In [2]:
import pandas as pd
import numpy as np
import microdf as mdf

c = pd.read_csv("../data/asec_2019_ipums.csv.gz")
c.columns = c.columns.str.lower()

c

,year,serial,month,cpsid,asecflag,asecwth,pernum,cpsidp,asecwt,age,...,offcutoff,cutoff,spmcaphous,spmwt,spmwic,spmheat,spmsnap,spmtotres,spmthresh,spmfamunit
0,2020,1,3,20190302844900,1,1552.90,1,20190302844901,1552.90,63,...,17120,17120,0.0,1552.90,0.0,0.0,0,86459.0,14700.0,1001
1,2020,1,3,20190302844900,1,1552.90,2,20190302844902,1552.90,67,...,17120,17120,0.0,1552.90,0.0,0.0,0,86459.0,14700.0,1001
2,2020,2,3,20181202843500,1,990.49,1,20181202843501,990.49,64,...,17120,17120,0.0,990.49,0.0,0.0,0,55275.0,14700.0,2001
3,2020,2,3,20181202843500,1,990.49,2,20181202843502,990.49,71,...,17120,17120,0.0,990.49,0.0,0.0,0,55275.0,14700.0,2001
4,2020,3,3,20190202831700,1,1505.27,1,20190202831701,1505.27,54,...,13300,13300,0.0,1505.27,0.0,0.0,0,28653.0,11920.0,3001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157954,2020,91498,3,0,1,444.18,1,0,444.18,72,...,12261,12261,0.0,444.18,0.0,0.0,0,42819.0,14770.0,91498001
157955,2020,91498,3,0,1,444.18,2,0,388.29,50,...,13300,13300,0.0,388.29,0.0,0.0,0,-3574.0,14770.0,91498003
157956,2020,91499,3,0,1,494.05,1,0,494.05,80,...,12261,12261,7042.0,494.05,0.0,200.0,0,20868.0,17800.0,91499001
157957,2020,91500,3,0,1,423.84,1,0,423.84,55,...,17622,17622,0.0,423.84,0.0,0.0,0,77447.0,27260.0,91500001


In [8]:
c.columns

Index(['year', 'serial', 'month', 'cpsid', 'asecflag', 'asecwth', 'pernum',
       'cpsidp', 'asecwt', 'age', 'race', 'inctot', 'offtotval', 'offcutoff',
       'cutoff', 'spmcaphous', 'spmwt', 'spmwic', 'spmheat', 'spmsnap',
       'spmtotres', 'spmthresh', 'spmfamunit'],
      dtype='object')

In [10]:
c[["cpsidp", "offcutoff"]].unique()

AttributeError: 'DataFrame' object has no attribute 'unique'

In [7]:
c.groupby("offcutoff").size().sort_index()

offcutoff
12261      7520
13300     18105
15453     13728
17120     17512
17555       140
17622      4172
19998     10284
20578     13035
20598      4389
25926     20400
26017      2336
26370      4816
26801      6720
30044       840
30510     10030
31275      4465
31800      1415
32263      2645
33522       294
34161      3756
35239      1968
35965      1878
36576       312
36721       798
36757        91
38262      1092
39635       707
40811       735
41442       483
41709        56
42066       384
42085        84
42348       245
43470       232
44818       304
45881       200
46630       176
47069        24
47485        56
49426       162
51406       245
51727       189
53025       252
54460       163
55503        93
56139        75
56621        19
56895        37
999999      297
dtype: int64

In [9]:
c.groupby("cutoff").size().sort_index()

cutoff
12261      7520
13300     18105
15453     13728
17120     17512
17555       140
17622      4172
19998     10284
20578     13035
20598      4389
25926     20400
26017      2336
26370      4816
26801      6720
30044       840
30510     10030
31275      4465
31800      1415
32263      2645
33522       294
34161      3756
35239      1968
35965      1878
36576       312
36721       798
36757        91
38262      1092
39635       707
40811       735
41442       483
41709        56
42066       384
42085        84
42348       245
43470       232
44818       304
45881       200
46630       176
47069        24
47485        56
49426       162
51406       245
51727       189
53025       252
54460       163
55503        93
56139        75
56621        19
56895        37
999999      297
dtype: int64